In [1]:
import pandas as pd
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import *
from tensorflow.keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [21]:
df = pd.read_csv('./crawling/news_dataconcat.csv', index_col = 0)
df = df.reset_index(drop=True)
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8029 entries, 0 to 8028
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     8029 non-null   object
 1   category  8029 non-null   object
dtypes: object(2)
memory usage: 125.6+ KB


In [22]:
X = df['title']
Y = df['category']

In [23]:
# 숫자로 된 라벨을 붙여주기 (라벨 엔코더)

encoder = LabelEncoder()
labeled_Y = encoder.fit_transform(Y)
label = encoder.classes_
print(label) # 0, 1, 2, 3, 4, 5 (알파벳 오름차순)

['Bond' 'China_Economy' 'Global_Economy' 'IB' 'Policy' 'Real_Estate'
 'Stock']


In [24]:
# one hot enconded 형식으로 바꿔 주었다

onehot_Y = to_categorical(labeled_Y)
print(onehot_Y)

[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


In [25]:
# 데이터타입이 시리즈이므로 바로 줄 수 없다
okt = Okt()
print(type(X))

# 문자열을 줘야 한다
okt_X = okt.morphs(X[0])
print(X[0])
print(okt_X)

<class 'pandas.core.series.Series'>
 KB금융  하반기부터 주주환원 개선 중간배당 등 검토   
['KB', '금융', '하반기', '부터', '주주', '환원', '개선', '중간배당', '등', '검토']


In [26]:
# 불용어 파일과 비교하여, 불용어 파일에 없으면 words 리스트에 추가하기

stopwords = pd.read_csv('../datasets/stopwords.csv')
words = []
for word in okt_X:
    if word not in list(stopwords['stopword']):
        words.append(word)
print(words)

['KB', '금융', '하반기', '주주', '환원', '개선', '중간배당', '검토']


In [27]:
# 형태소별로 분석된 것의 리스트가 시리즈인 X의 요소로

for i in range(len(X)):
    X[i] = okt.morphs(X[i])
print(X)

0              [KB, 금융, 하반기, 부터, 주주, 환원, 개선, 중간배당, 등, 검토]
1                             [은, 성수, 공매도, 비판, 달, 게, 받겠다]
2       [작년, 하반기, 우수, 국고, 채, PD, 5, 개사, 선정, 종합, 1, 위, ...
3        [apos, 사외이사, 추천, apos, 본격, 화하는, 국민연금, 다음, 조치, 는]
4            [사학, 연금, 퇴직, 급여, 사전, 청구, 제도, 실시, 급여, 조기, 지급]
                              ...                        
8024    [홍대, 상권, 도, 코로나, 19, 직격탄, 카페, 월, 매출, 작년, 말, 보다...
8025               [국토, 硏, 거래, 늘어난다고, 아파트, 값, 이, 오르진, 않아]
8026           [현대, ENG, 대만, 서, 4천억, 규모, 발전소, 증설, 공사, 수주]
8027    [5년, 만에, 재건축, 수, 주전, 뛰어든, 삼성, 물산, 계, 열사, 총, 결집...
8028    [LH, 신혼, 희망, 타운, 올해, 8월, 부터, 공급, 12월, 71, 물량, 집중]
Name: title, Length: 8029, dtype: object


In [28]:
# j: X의 요소 개수까지
# i: X의 한 요소 내 요소 개수까지
# 한 글자 이상 and 불용어 리스트에 없으면 -
# 하나의 문장으로 바꾸기(토크나이저에 주기 위해)

for j in range(len(X)):
    results = []
    for i in range(len(X[j])):
        if (X[j][i] not in list(stopwords['stopword'])):
            # (len(X[j][i]) > 1) and 
            results.append(X[j][i])
    X[j] = ' '.join(results) 
print(X)

0                  KB 금융 하반기 주주 환원 개선 중간배당 검토
1                         은 성수 공매도 비판 달 게 받겠다
2       작년 하반기 우수 국고 채 PD 5 개사 선정 종합 1 위 KB 證
3          apos 사외이사 추천 apos 본격 화하는 국민연금 조치 는
4            사학 연금 퇴직 급여 사전 청구 제도 실시 급여 조기 지급
                        ...                  
8024      홍대 상권 도 코로나 19 직격탄 카페 매출 작년 말 708만원
8025               국토 硏 거래 늘어난다고 아파트 값 오르진 않아
8026          현대 ENG 대만 서 4천억 규모 발전소 증설 공사 수주
8027      5년 만에 재건축 주전 뛰어든 삼성 물산 계 열사 총 결집 협업
8028        LH 신혼 희망 타운 올해 8월 공급 12월 71 물량 집중
Name: title, Length: 8029, dtype: object


In [29]:
# 형태소를 숫자로 바꿔 토크나이저에 넘겨주어야 한다
# token.fit_on_texts: 단어 사전 만들기
# token.texts_to_sequences: 문장의 순서를 유지(순서 정보가 중요)

token = Tokenizer()
token.fit_on_texts(X)
tokened_X = token.texts_to_sequences(X)
print(tokened_X)

[[130, 25, 701, 769, 3591, 171, 2887, 291], [15, 736, 346, 1687, 308, 1529, 5123], [42, 701, 1400, 190, 156, 2446, 45, 1283, 297, 41, 3, 146, 130, 79], [1, 1876, 523, 1, 347, 2888, 359, 448, 60], [1877, 468, 985, 1878, 645, 1187, 600, 601, 1878, 737, 434], [1, 3592, 5124, 1, 624, 3593, 416, 1284, 30, 2889, 1, 2890, 1, 125], [130, 25, 42, 147, 121, 8, 1688, 19, 292, 48, 8], [34, 122, 927, 3594, 5125, 1187, 2447], [1689, 68, 574, 372, 269, 1285, 2448, 1879, 5126, 172], [25, 2891, 204, 5127, 1401, 157, 3595, 60], [390, 175, 241, 417, 1690, 7, 986, 702, 524, 1530, 123, 5128, 172], [2102, 506, 5129, 737, 5130, 2103, 1120, 3596, 2104], [485, 1402, 122, 646, 486, 107, 214, 1403, 1531, 123, 1, 1], [2105, 5131, 149, 575, 94, 160, 602, 5132, 12], [166, 163, 95, 2892, 987, 102, 108, 5133, 330], [86, 3597, 807], [2106, 869, 2893, 2449, 434, 3598, 323, 20], [65, 15, 171, 5134, 324, 461, 135, 647, 1188, 1532, 1, 1], [34, 122, 673, 12, 648, 104, 404, 337, 1121, 487, 601], [3599, 2894, 32, 2895, 1404,

In [30]:
# fit_on_texts한 토큰 저장

import pickle
with open('./news_token.pickle', 'wb') as f:
    pickle.dump(token, f)

In [31]:
# 패딩을 위한 0을 포함한(+1) 단어의 개수
wordsize = len(token.word_index) + 1 
print(wordsize)

9656


In [32]:
'''
문장의 길이를 맞춰주는 작업; 입력 레이어의 사이즈가 동일해야 하므로
제일 긴 문장을 기준으로, 모자라는 부분은 0으로 채운다
따라서 제일 긴 문장을 먼저 찾아야 한다

0으로 시작하는 max가 tokened_X의 어떤 요소의 개수보다 작으면 그 요소의 개수와 동일하게 만든다
'''

max = 0
for i in range(len(tokened_X)):
    if max < len(tokened_X[i]):
        max = len(tokened_X[i])
print(max)

17


In [33]:
# max에 맞춰 0으로 채워준다

X_pad = pad_sequences(tokened_X, max)
print(X_pad[:10])

[[   0    0    0    0    0    0    0    0    0  130   25  701  769 3591
   171 2887  291]
 [   0    0    0    0    0    0    0    0    0    0   15  736  346 1687
   308 1529 5123]
 [   0    0    0   42  701 1400  190  156 2446   45 1283  297   41    3
   146  130   79]
 [   0    0    0    0    0    0    0    0    1 1876  523    1  347 2888
   359  448   60]
 [   0    0    0    0    0    0 1877  468  985 1878  645 1187  600  601
  1878  737  434]
 [   0    0    0    1 3592 5124    1  624 3593  416 1284   30 2889    1
  2890    1  125]
 [   0    0    0    0    0    0  130   25   42  147  121    8 1688   19
   292   48    8]
 [   0    0    0    0    0    0    0    0    0    0   34  122  927 3594
  5125 1187 2447]
 [   0    0    0    0    0    0    0 1689   68  574  372  269 1285 2448
  1879 5126  172]
 [   0    0    0    0    0    0    0    0    0   25 2891  204 5127 1401
   157 3595   60]]


In [34]:
# train(훈련), test(검증) set

X_train, X_test, Y_train, Y_test = train_test_split(X_pad, onehot_Y, test_size=0.1)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(7226, 17)
(803, 17)
(7226, 7)
(803, 7)


In [35]:
# 저장

xy = X_train, X_test, Y_train, Y_test
np.save('./news_data_max_17_size_9656.npy', xy)

C:\Users\698\anaconda3\envs\AI_dev_env\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
